In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyneb as pn

## The EmissionLine class

This is the class characterizing emission lines. An emission line is identified by an element and a spectrum (which identify the emitting ion), a wavelength in Angstrom, a blend flag, a label in the standard PyNeb format, an observed intensity, a reddening-corrected intensity, an expression describing how the intensity depends on the included wavelengths, an observational error and an error on the corrected intensity. Other programs determine one or more of these values.

To instantiate an Emission Line object, use the following:

In [2]:
line = pn.EmissionLine('O', 3, 5007, obsIntens=[1.4, 1.3])
line2 = pn.EmissionLine(label = 'O3_5007A', obsIntens=320, corrected=True)

obsIntens is a value, a list or a numpy array of values corresponding to the observed intensity of the given emission line.

In [3]:
print(line)

Line O3 O3_5007A


To know how the label of a given line is exactly spelled, you can print the dictionary pn.LINE_LABEL_LIST

In [4]:
print(pn.LINE_LABEL_LIST['O3'])

['4931A', '4959A', '5007A', '2315A', '2321A', '2331A', '4363A', '1658A', '1661A', '1666A', '2497A', '5833A', '88.3m', '32.6m', '51.8m']


It is possible to instantiate a line not contained in the pn.LINE_LABEL_LIST. In this case a warning is issued, but the code doesn't stop.

The observed intensity is stored in __line.obsIntens__ and the extinction-corrected intensity is stored in __line.corrIntens__. __line.corrIntens__ is set to 0.0 when the line is instantiated, unless the parameter corrected is set to __True__, in which case the observed value __obsIntens__ is copied to the __corrIntens__ slot (the same applies for __corrError__, which is set to __obsError__). 

The __corrIntens__ value can also be computed using an instantiation of the __pn.RedCorr__ class:

In [5]:
redcorr = pn.RedCorr(E_BV = 0.87, law = 'F99')

In [6]:
line.correctIntens(redcorr) #redcorr is used to compute line.corrIntens

The line information is printed using:

In [7]:
line.printLine()

Line O3 O3_5007A evaluated as L(5007)
Observed intensity: [ 1.4  1.3]
Observed error: [ 0.  0.]
Corrected intensity: [ 22.58352855  20.97041937]
Corrected error: [ 0.  0.]


Most of the times, users will not need to define or manipulate EmissionLine objects, since most of the work on the EmissionLine objects will be performed from the Observation class (read data, extinction correction); see next section.

WARNING: Note that the wavelengths assigned to EmissionLine objects are simply the numerical part of the label:

In [9]:
Hb1 = pn.EmissionLine(label='H1r_4861A').wave
print(Hb1)

4861.0


whereas pn.Atom computes them as the difference from energy levels:

In [10]:
Hb2 = pn.RecAtom('H', 1).getWave(4, 2)
print(Hb2)

4861.33165987


This can cause small errors when both methods are used simultaneously. For instance, the extinction correction at Hb1 is slightly different from the expected value of 1: 

In [13]:
rc = pn.RedCorr() 
rc.E_BV = 1.34
rc.law = 'F99'
rc.getCorrHb(Hb1)

1.000410798530041

This happens because the ExtCorr uses the precise H$\beta$ value computed from energy levels. If this roundoff error exceeds your tolerance, a possible workaround is forcing the emission line to have exactly the wavelength computed from the energy levels:

In [16]:
O3w = pn.EmissionLine('O', 3, wave=5007).wave
print(rc.getCorrHb(O3w))

0.836202049828


In [18]:
Hb11 = pn.EmissionLine('H', 1, wave=Hb2).wave
print(rc.getCorrHb(Hb11)) 
# This will generate a warning (as the transition is not included in the inventory for the specified atom), 
# but the code won't stop.

warng EmissionLine: Atom H1 not valid
1.0


## The Observation class: reading and dealing with observations

### Reading observation from a file

pn.Observation is the class characterizing observation records. An observation record is composed of an object identifier, the observed intensity of one or more emission lines, and, optionally, the dereddened line intensities and the identifier of the extinction law used, and the value of c(Hbeta).

Observations can be initialized by reading data files, which can be organized with different emission lines either in rows or columns (usually, in a survey of many objects with few emission lines emission lines change across columns; and in a high-resolution observation of a small sample of objects lines change across rows).

The following is an example of how to define an observation:

In [21]:
obs = pn.Observation()

In [22]:
%%writefile observations1.dat
LINE SMC_24
S4_10.5m   7.00000
Ne2_12.8m  8.3000
Ne3_15.6m 34.10
S3_18.7m  10.
O2_3726A  39.700
O2_3729A  18.600
Ne3_3869A 18.90
Ne3_3968A  6.4
S2_4069A   0.85
S2_4076A   0.450
O3_4363A   4.36
H1r_4861A 100.00
O3_5007A 435.09
N2_5755A   0.510000
S3_6312A   0.76
O1_6300A   1.69
O1_6364A   0.54
N2_6548A   6.840000
H1r_6563A   3.45
N2_6584A  19.00
S2_6716A   1.220000
S2_6731A   2.180000
Ar3_7136A  4.91
O2_7319A+   6.540000
O2_7330A+   5.17

Overwriting observations1.dat


In [23]:
obs.readData('observations1.dat', fileFormat='lines_in_rows', err_default=0.05) # fill obs with data read from smc_24.dat

In [24]:
obs.printIntens(returnObs=True)

S4_10.5m [ 7.]
Ne2_12.8m [ 8.30000019]
Ne3_15.6m [ 34.09999847]
S3_18.7m [ 10.]
O2_3726A [ 39.70000076]
O2_3729A [ 18.60000038]
Ne3_3869A [ 18.89999962]
Ne3_3968A [ 6.4000001]
S2_4069A [ 0.85000002]
S2_4076A [ 0.44999999]
O3_4363A [ 4.36000013]
H1r_4861A [ 100.]
O3_5007A [ 435.08999634]
N2_5755A [ 0.50999999]
S3_6312A [ 0.75999999]
O1_6300A [ 1.69000006]
O1_6364A [ 0.54000002]
N2_6548A [ 6.84000015]
H1r_6563A [ 3.45000005]
N2_6584A [ 19.]
S2_6716A [ 1.22000003]
S2_6731A [ 2.18000007]
Ar3_7136A [ 4.90999985]
O2_7319A+ [ 6.53999996]
O2_7330A+ [ 5.17000008]


In [25]:
obs.extinction.law = 'CCM89'  # define the extinction law from Cardelli et al.
obs.correctData()                      # the dereddened data are computed

The data can be read by the readData method as above or directly while instantiating the object:

In [26]:
obs = pn.Observation('observations1.dat', fileFormat='lines_in_rows', corrected=True)

The format of the data file from which the emission line intensities are read can be one of three kinds: "lines_in_rows" as above, or "lines_in_cols" like this one:

In [27]:
%%writefile observations2.dat
NAME O2_3726A  O2_3726Ae O2_3729A O2_3729Ae
NGC3132 0.93000   0.05000   0.17224200 0.10  
IC418 1.28000   0.05000   0.09920000 0.05 
M33 0.03100   0.080     0.03100    0.10

Writing observations2.dat


In [28]:
obs2 = pn.Observation('observations2.dat', fileFormat='lines_in_cols', corrected=True)

or fileFormat='lines_in_rows_err_cols' (errors labeled “err”. Don’t name an observation “err”!) like this one:

In [29]:
%%writefile observations3.dat
LINE     TT   err  TT2 err TT3 err
cHbeta   1.2  0.0  1.5 0.2 1.1 0.2
O3_5007A 1.5  0.15 1.3  .2 1.1 0.1
H1_6563A 2.89 0.05 1.6 0.3 1.3 0.1
N2_6584A 1.   0.20 0.3 0.5 1.5 0.1

Writing observations3.dat


In [31]:
#obs3 = pn.Observation('observations3.dat', fileFormat='lines_in_rows_err_cols', corrected=False)

The delimiter between the columns is any sequence of spaces or TAB, but it can be changed using the delimiter parameter. The line names are defined by a label starting with the name of the atom (‘O2’), followed by an underscore, followed by a wavelength and ending with a unit (‘A’ or ‘m’). The list of all the lines managed by PyNeb, ordered by atoms, is obtained by entering:

In [32]:
for atom in pn.LINE_LABEL_LIST:
    print(atom, pn.LINE_LABEL_LIST[atom])

H1r ['1216A', '1026A', '973A', '6563A', '4861A', '4341A', '4102A', '3970A', '3889A', '3835A', '3798A', '1.87m', '1.28m', '1.09m', '9546A', '9229A']
He1r ['5876A', '2945A', '3188A', '3614A', '3889A', '3965A', '4026A', '4121A', '4388A', '4438A', '4471A', '4713A', '4922A', '5016A', '5048A', '5876A', '6678A', '7065A', '7281A', '9464A', '10830A', '11013A', '11969A', '12527A', '12756A', '12785A', '12790A', '12846A', '12968A', '12985A', '13412A', '15084A', '17003A', '18556A', '18685A', '18697A', '19089A', '19543A', '20425A', '20581A', '20602A', '21120A', '21132A', '21608A', '21617A']
He2r ['1640A', '1215A', '1084A', '4686A', '3203A', '6560A', '5411A', '4859A']
3He2 ['3.50c']
Al2 ['2674A', '2670A', '2661A', '1671A', '4451A', '4463A', '4488A', '164.2m', '54.1m', '80.7m']
Ar2 ['7.0m']
Ar3 ['7136A', '7751A', '8036A', '3005A', '3109A', '3154A', '5192A', '9.0m', '6.37m', '21.8m']
Ar4 ['4740A', '4711A', '2868A', '7263A', '7332A', '2854A', '7170A', '7237A', '77.4m', '56.2m']
Ar5 ['6133A', '6435A', '7

The presence of a trailing “e” at the end of the label points to the error associated to the line. The error is considered to be relative to the intensity (i.e., 0.05 means 5% of the intensity), unless the parameter errIsRelative is set to False. A common value for all the errors can be defined by the parameter __err_default__ (0.10 is the default value).

### Extinction correction in Observation class

Once the data have been read, they have to be corrected from extinction. An instantiation of __RedCorr()__ is available inside the __Observation__ object as __obs.extinction__.

If the data file contains __cHbeta__ or __E(B-V)__ alongside of line labels, the corresponding information on extinction is transmitted to the extinction correction object. Otherwise, the extinction parameters must be set manually; for example:

In [33]:
obs = pn.Observation('observations1.dat', fileFormat='lines_in_rows', corrected=True)
obs.extinction.cHbeta = 1.2
obs.extinction.E_BV = 0.34

An extinction law has to be specified in either case:

In [34]:
obs.extinction.law = 'F99'

To correct all the lines at once:

In [36]:
obs.correctData()

In [37]:
obs.printIntens(returnObs=True)

S4_10.5m [ 7.]
Ne2_12.8m [ 8.30000019]
Ne3_15.6m [ 34.09999847]
S3_18.7m [ 10.]
O2_3726A [ 39.70000076]
O2_3729A [ 18.60000038]
Ne3_3869A [ 18.89999962]
Ne3_3968A [ 6.4000001]
S2_4069A [ 0.85000002]
S2_4076A [ 0.44999999]
O3_4363A [ 4.36000013]
H1r_4861A [ 100.]
O3_5007A [ 435.08999634]
N2_5755A [ 0.50999999]
S3_6312A [ 0.75999999]
O1_6300A [ 1.69000006]
O1_6364A [ 0.54000002]
N2_6548A [ 6.84000015]
H1r_6563A [ 3.45000005]
N2_6584A [ 19.]
S2_6716A [ 1.22000003]
S2_6731A [ 2.18000007]
Ar3_7136A [ 4.90999985]
O2_7319A+ [ 6.53999996]
O2_7330A+ [ 5.17000008]


In [38]:
obs.printIntens()

S4_10.5m [ 7.11975803]
Ne2_12.8m [ 8.41496764]
Ne3_15.6m [ 34.48348489]
S3_18.7m [ 10.09307839]
O2_3726A [ 171.24206763]
O2_3729A [ 80.15612017]
Ne3_3869A [ 78.13399959]
Ne3_3968A [ 25.71732799]
S2_4069A [ 3.32014218]
S2_4076A [ 1.75430783]
O3_4363A [ 15.7036518]
H1r_4861A [ 310.25435475]
O3_5007A [ 1289.85131043]
N2_5755A [ 1.24390974]
S3_6312A [ 1.66198348]
O1_6300A [ 3.70383903]
O1_6364A [ 1.16980719]
N2_6548A [ 14.34619111]
H1r_6563A [ 7.21747442]
N2_6584A [ 39.60657366]
S2_6716A [ 2.48780908]
S2_6731A [ 4.43455095]
Ar3_7136A [ 9.38391466]
O2_7319A+ [ 12.17965989]
O2_7330A+ [ 9.61370718]


If you want the corrected line intensities to be normalized to a given wavelength, use the following:

In [39]:
obs.correctData(normWave=4861.)

The extinction correction can be determined by comparing the observed values to a theoretical ratio, as in the following:

In [41]:
obs.printIntens()

S4_10.5m [ 2.29481325]
Ne2_12.8m [ 2.71228027]
Ne3_15.6m [ 11.11458529]
S3_18.7m [ 3.25316252]
O2_3726A [ 55.19408995]
O2_3729A [ 25.83561486]
Ne3_3869A [ 25.18385266]
Ne3_3968A [ 8.28911105]
S2_4069A [ 1.07013556]
S2_4076A [ 0.56544181]
O3_4363A [ 5.06154114]
H1r_4861A [ 100.]
O3_5007A [ 415.73995358]
N2_5755A [ 0.40093224]
S3_6312A [ 0.53568417]
O1_6300A [ 1.19380727]
O1_6364A [ 0.37704779]
N2_6548A [ 4.62400959]
H1r_6563A [ 2.32630882]
N2_6584A [ 12.76583972]
S2_6716A [ 0.80186113]
S2_6731A [ 1.42932755]
Ar3_7136A [ 3.02458757]
O2_7319A+ [ 3.92570151]
O2_7330A+ [ 3.09865342]


In [42]:
obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.85)
print(obs.extinction.E_BV)
obs.correctData(normWave=4861.)

[-3.80825098]


In [43]:
obs.printIntens()

S4_10.5m [ 1862873.3321508]
Ne2_12.8m [ 2289624.63614709]
Ne3_15.6m [ 9681842.82513927]
S3_18.7m [ 2900918.21157661]
O2_3726A [ 0.99069915]
O2_3729A [ 0.46892362]
Ne3_3869A [ 0.75888083]
Ne3_3968A [ 0.35320246]
S2_4069A [ 0.06443513]
S2_4076A [ 0.03486379]
O3_4363A [ 0.81982369]
H1r_4861A [ 100.]
O3_5007A [ 724.22886843]
N2_5755A [ 7.55158652]
S3_6312A [ 38.21722558]
O1_6300A [ 82.92131172]
O1_6364A [ 30.17724087]
N2_6548A [ 549.03539807]
H1r_6563A [ 285.]
N2_6584A [ 1633.68483422]
S2_6716A [ 134.21899293]
S2_6731A [ 246.50683125]
Ar3_7136A [ 1116.57610378]
O2_7319A+ [ 1987.64418295]
O2_7330A+ [ 1598.12655872]


By default, this method prints out the corrected intensities. To print the observed intensities, use the __returnObs=True__ parameter.

The method __getSortedLines__ returns the lines sorted in alphabetical order according to either the emitting atoms (default) or the wavelength (using the __crit='wave'__ parameter):

In [45]:
for line in obs.getSortedLines(): 
    print(line.label, line.corrIntens[0])

Ar3_7136A 1116.57610378
H1r_4861A 100.0
H1r_6563A 285.0
N2_5755A 7.55158651669
N2_6548A 549.03539807
N2_6584A 1633.68483422
Ne2_12.8m 2289624.63615
Ne3_15.6m 9681842.82514
Ne3_3869A 0.758880826821
Ne3_3968A 0.35320245771
O1_6300A 82.921311722
O1_6364A 30.1772408716
O2_3726A 0.990699145083
O2_3729A 0.468923618506
O2_7319A+ 1987.64418295
O2_7330A+ 1598.12655872
O3_4363A 0.819823693016
O3_5007A 724.228868428
S2_4069A 0.0644351280132
S2_4076A 0.0348637946044
S2_6716A 134.218992935
S2_6731A 246.506831252
S3_18.7m 2900918.21158
S3_6312A 38.2172255813
S4_10.5m 1862873.33215


The following method, which gives the list of all the atoms implied in the observed emission lines, will be useful later:

In [46]:
atomList = obs.getUniqueAtoms()

In [47]:
atomList

array(['Ar3', 'H1r', 'N2', 'Ne2', 'Ne3', 'O1', 'O2', 'O3', 'S2', 'S3', 'S4'], 
      dtype='<U3')

### Adding observations and lines

Once an __Observation__ object is instantiated, you can add a new observation (corresponding, e.g., to a new object or a new fiber) by using:

In [48]:
obs.addObs('test', np.random.rand(25))

where ‘test’ is the name of the new observation. The new observation must have the same size of __obs__, that is, it must contain __obs.n_lines__ lines.

In [50]:
obs.printIntens()

S4_10.5m [  1.86287333e+06   8.17609704e-01]
Ne2_12.8m [  2.28962464e+06   4.84209585e-01]
Ne3_15.6m [  9.68184283e+06   5.69878150e-01]
S3_18.7m [  2.90091821e+06   1.69287051e-01]
O2_3726A [ 0.99069915  0.27622725]
O2_3729A [ 0.46892362  0.93353075]
Ne3_3869A [ 0.75888083  0.76410908]
Ne3_3968A [ 0.35320246  0.75877006]
S2_4069A [ 0.06443513  0.43461895]
S2_4076A [ 0.03486379  0.01242744]
O3_4363A [ 0.81982369  0.87784109]
H1r_4861A [ 100.            0.10581813]
O3_5007A [  7.24228868e+02   2.66732250e-02]
N2_5755A [ 7.55158652  0.26535309]
S3_6312A [ 38.21722558   0.85286266]
O1_6300A [ 82.92131172   0.16316097]
O1_6364A [ 30.17724087   0.30763595]
N2_6548A [  5.49035398e+02   3.34564023e-01]
H1r_6563A [ 285.            0.60962985]
N2_6584A [  1.63368483e+03   7.55564203e-01]
S2_6716A [  1.34218993e+02   1.14665408e-02]
S2_6731A [ 246.50683125    0.55873937]
Ar3_7136A [  1.11657610e+03   5.42686282e-01]
O2_7319A+ [  1.98764418e+03   8.20687911e-01]
O2_7330A+ [  1.59812656e+03   9.84

In [51]:
line = pn.EmissionLine(label='Cl3_5518A', obsIntens=[3.5, 2.5])
obs.addLine(line)

In [52]:
obs.printIntens()

S4_10.5m [  1.86287333e+06   8.17609704e-01]
Ne2_12.8m [  2.28962464e+06   4.84209585e-01]
Ne3_15.6m [  9.68184283e+06   5.69878150e-01]
S3_18.7m [  2.90091821e+06   1.69287051e-01]
O2_3726A [ 0.99069915  0.27622725]
O2_3729A [ 0.46892362  0.93353075]
Ne3_3869A [ 0.75888083  0.76410908]
Ne3_3968A [ 0.35320246  0.75877006]
S2_4069A [ 0.06443513  0.43461895]
S2_4076A [ 0.03486379  0.01242744]
O3_4363A [ 0.81982369  0.87784109]
H1r_4861A [ 100.            0.10581813]
O3_5007A [  7.24228868e+02   2.66732250e-02]
N2_5755A [ 7.55158652  0.26535309]
S3_6312A [ 38.21722558   0.85286266]
O1_6300A [ 82.92131172   0.16316097]
O1_6364A [ 30.17724087   0.30763595]
N2_6548A [  5.49035398e+02   3.34564023e-01]
H1r_6563A [ 285.            0.60962985]
N2_6584A [  1.63368483e+03   7.55564203e-01]
S2_6716A [  1.34218993e+02   1.14665408e-02]
S2_6731A [ 246.50683125    0.55873937]
Ar3_7136A [  1.11657610e+03   5.42686282e-01]
O2_7319A+ [  1.98764418e+03   8.20687911e-01]
O2_7330A+ [  1.59812656e+03   9.84

### Getting line intensities

You can extract the line intensities from an __Observation__ object by, for example:

In [53]:
obs.names

['SMC_24', 'test']

In [54]:
obs.getIntens(obsName='SMC_24')

{'Ar3_7136A': 1116.5761037772124,
 'Cl3_5518A': 8.8584535640522582e-05,
 'H1r_4861A': 100.0,
 'H1r_6563A': 285.0000000000004,
 'N2_5755A': 7.5515865166910849,
 'N2_6548A': 549.03539806966057,
 'N2_6584A': 1633.6848342208623,
 'Ne2_12.8m': 2289624.6361470865,
 'Ne3_15.6m': 9681842.8251392711,
 'Ne3_3869A': 0.75888082682114033,
 'Ne3_3968A': 0.35320245770990177,
 'O1_6300A': 82.921311721997114,
 'O1_6364A': 30.177240871552701,
 'O2_3726A': 0.99069914508263235,
 'O2_3729A': 0.46892361850581255,
 'O2_7319A+': 1987.6441829484479,
 'O2_7330A+': 1598.1265587226821,
 'O3_4363A': 0.81982369301604263,
 'O3_5007A': 724.22886842764478,
 'S2_4069A': 0.064435128013150128,
 'S2_4076A': 0.034863794604422474,
 'S2_6716A': 134.21899293495827,
 'S2_6731A': 246.50683125182974,
 'S3_18.7m': 2900918.2115766057,
 'S3_6312A': 38.217225581304128,
 'S4_10.5m': 1862873.3321507953}

In [55]:
obs.getIntens()['O2_7330A+']

array([  1.59812656e+03,   9.84102059e-01])

## Using Observation to determine ionic abundances

Once the electron temperature and density are determined, it is easy to obtain the ionic abundances from a set of emission lines included in an __Observation__ object:

In [56]:
obs = pn.Observation()
obs.readData('observations1.dat', fileFormat='lines_in_rows', err_default=0.05) # fill obs with data read from observations1.dat
obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.85)
obs.correctData(normWave=4861.)
Te = [10000.]
Ne = [1e3]
# Define a dictionary to hold all the Atom objects needed
all_atoms = pn.getAtomDict(atom_list=obs.getUniqueAtoms())
# define a dictionary to store the abundances
ab_dict = {}
# we  use the following lines to determine the ionic abundances
ab_labels = ['N2_6584A', 'O2_3726A', 'O3_5007A', 'S2_6716A', 
             'S3_6312A', 'Ar3_7136A', 'Ne3_3869A']
for line in obs.getSortedLines():
    if line.label in ab_labels:
        ab = all_atoms[line.atom].getIonAbundance(line.corrIntens, Te, Ne, 
                                                  to_eval=line.to_eval, Hbeta=100)
        ab_dict[line.atom] = ab

warng _ManageAtomicData: rec data not available for Ar3
warng _ManageAtomicData: atom data not available for H1
warng _ManageAtomicData: coll data not available for H1
warng _ManageAtomicData: rec data not available for Ne2
warng _ManageAtomicData: rec data not available for Ne3
warng _ManageAtomicData: rec data not available for S2
warng _ManageAtomicData: rec data not available for S3
warng _ManageAtomicData: rec data not available for S4


In [57]:
ab_dict

{'Ar3': array([  4.06352863e-07]),
 'N2': array([  3.91252886e-06]),
 'Ne3': array([  2.05860787e-05]),
 'O2': array([  3.10454298e-05]),
 'O3': array([ 0.00015164]),
 'S2': array([  8.23174570e-08]),
 'S3': array([  1.69309386e-06])}